1\. Write a function that converts number representation (bin<->dec<->hex)

In [6]:
import string 

def nbase_to_dec(strnbase, base):
    # remove the not useful part of the string an save the sign if negative
    if strnbase[0] == '-':
        try: 
            # check if you can convert the second number to integer
            # if so, it is not 'x' or 'b', so you don't have to remove the first 2 digits
            int(strnbase[2])  
            strnbase = strnbase[1:] # remove the sign from the string
        except:
            strnbase = strnbase[3:]
        sign = '-'
    else:
        try: 
            # check if you can convert the second number to integer
            # if so, it is not 'x' or 'b', so you don't have to remove the first 2 digits
            int(strnbase[1])  
        except: 
            strnbase = strnbase[2:]
        sign = '+'
    
    # invert the string
    strnbase = strnbase[::-1]    
    # start from 0
    ndec = 0
    # algorithm
    for i in range(len(strnbase)):
        try: 
            int(strnbase[i])
            ndec += int(strnbase[i])*base**i
        except:
            value = string.ascii_lowercase.find(strnbase[i])+10
            ndec += value*base**i
    # sign
    if sign == '-':
        ndec*=-1

    return ndec

def dec_to_nbase(ndec, base):
    basedic = {2: 'b', 16: 'x', 'other': 'o'}
    
    if base == 2 or base == 16:
        basechar = basedic[base]
    else:
        basechar = basedic['other']
    
    strbase = ''
        
    # save the sign and convert the number to positive
    sign = ''
    if ndec < 0:
        sign='-'
        ndec*=-1
    
    while ndec>=1:
        rest = ndec%base
        if rest > 9:
            rest = string.ascii_lowercase[rest-10]
        else: rest = str(rest)
        strbase = rest+strbase
        ndec = ndec//base
            
    return sign+'0'+basechar+strbase


In [7]:
# TEST THE FUNCTIONS

a = 274

print('Starting number:', a)
a_bin = bin(a)
a_hex = hex(a)

print('\nBinary:')
print('With my func:', dec_to_nbase(a, 2))
print('With py func:', a_bin)

print('\nBack to the starting decimal')
print('With my func:', nbase_to_dec(a_bin, 2))
print('With py func:', int(a_bin, 2))

print('\nHexadecimal:')
print('With my func:', dec_to_nbase(a, 16))
print('With py func:', a_hex)

print('\nBack to the starting decimal')
print('With my func:', nbase_to_dec(a_hex, 16))
print('With py func:', int(a_hex, 16))

Starting number: 274

Binary:
With my func: 0b100010010
With py func: 0b100010010

Back to the starting decimal
With my func: 274
With py func: 274

Hexadecimal:
With my func: 0x112
With py func: 0x112

Back to the starting decimal
With my func: 274
With py func: 274


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [8]:
def singleprecision(nbin):
    try: 
            # check if you can convert the second number to integer
            # if so, it is not 'x' or 'b', so you don't have to remove the first 2 digits
            int(nbin[1])  
    except: 
        nbin = nbin[2:]
        
    if len(nbin)!=32:
        return 'Not a 32 bit word'
    
    bias = 127
    nbin = [int(i) for i in nbin]
    
    # in the 0 python position I have the 31th bit
    e = nbase_to_dec(nbin[1:9], 2)
    
    f = 0
    for i in range(23):
        f += nbin[9+i]*2**(-i-1)
    
    return (-1)**(nbin[0])*(1+f)*2**(e-bias)


test_1 = '00000011111000000000000000000000'
test_2 = '0b00000011111000000000000000000000'
test_3 = '11000000101100000000000000000000'
test_4 = '010101'
print(singleprecision(test_1))
print(singleprecision(test_2))
print(singleprecision(test_3))
print(singleprecision(test_4))


1.316553672920962e-36
1.316553672920962e-36
-5.5
Not a 32 bit word


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits. Within a factor of 2 means no more than twice and no less the number found. 

In [2]:
import math as m

In [10]:
# Observe the differences:

n1 = 1.7976931348623157e+308
n2 = 1.8e+500
n3 = 2e+500
n4 = 2*10**500
n5 = 2.0*10**308      
# n6 = 2.0*10**309      # uncomment to get the overflow message
print('1', n1)
print('2', n2)
print('3', n3)
print('4', n4)
print('5', n5)
# print('6', n6)        # uncomment to get the overflow message

print('\nTest 1')
test = 1.0
for i in range(10):
    test*= (2**200)
    print(test)

# uncomment to get the overflow message
# print('\nTest 2')
# for i in range(10):
#     test = 2.0*10**(200*i)
#     print(test)

1 1.7976931348623157e+308
2 inf
3 inf
4 200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
5 inf

Test 1
1.6069380442589903e+60
2.5822498780869086e+120
4.149515568880993e+180
6.668014432879854e+240
1.0715086071862673e+301
inf
inf
inf
inf
inf


In [42]:
# SOLUTION OF THE EXERCISE
# In this case:
# - overflow are sets to --> inf
# - underflow are sets to --> 0

x_over = 1.0
x_under = 1.0

is_overflow = False
is_underflow = False

# speed of the algorithm 
# power_over = 200
# power_under = 200
# scale = 2

while is_overflow == False or is_underflow == False:
    tmp = x_over * 2
    if m.isinf(tmp):
        is_overflow = True
    else: 
        x_over = tmp
        
    tmp = x_under/2
    if tmp == 0:
        is_underflow = True
    else: 
        x_under = tmp
        
print('Overflow limit:', x_over)
print('Underflow limit:', x_under)

# check
print('\nOverflow?', x_over*2)
print('Underflow?', x_under/2)

Overflow limit: 8.98846567431158e+307
Underflow limit: 5e-324

Overflow? inf
Underflow? 0.0


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [41]:
n = 2
precision = 1

are_equals = False
while are_equals == False:
    precision /= 2
    new_n = n + precision
    
    if n == new_n:
        are_equals = True
        print('They are equals!')
        
print('Machine precision:', precision)
    

They are equals!
Machine precision: 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

In [59]:
def quadratic_equation(a, b, c):
    x1 = (-b+m.sqrt(b*b-4*a*c))/(2*a)
    x2 = (-b-m.sqrt(b*b-4*a*c))/(2*a) 
    return [x1, x2]

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

In [76]:
x_12 = quadratic_equation(0.001, 1000, 0.001)
print('Solutions: \nx1 = %f \nx2 = %f' %(x_12[0], x_12[1]))


Solutions: 
x1 = -0.000001 
x2 = -999999.999999


(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. $
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}\cdot\frac{-b\mp\sqrt{b^2-4ac}}{-b\mp\sqrt{b^2-4ac}} = \frac{4ac}{2a(-b\mp\sqrt{b^2-4ac})} = \frac{2c}{-b\mp\sqrt{b^2-4ac}}
$

In [77]:
def quadratic_equation_new(a, b, c):
    x1 = 2*c/(-b-m.sqrt(b*b-4*a*c))
    x2 = 2*c/(-b+m.sqrt(b*b-4*a*c)) 
    
    return [x1, x2]

x_12 = quadratic_equation_new(0.001, 1000, 0.001)
print('Solutions: \nx1 = %f \nx2 = %f' %(x_12[0], x_12[1]))

Solutions: 
x1 = -0.000001 
x2 = -1000010.575513


_How does it compare with what previously obtained? Why?_ <br>
While for $x_1$ I get the same result with the different formulas, $x_2$ is slightly different in the two cases and the solution is correct only with the first method. This is due to the fact that, using the second formula, I'm taking the difference between 2 very close numbers $(-b+\sqrt{b^2-4ac})$ and then divide $c$ by it. In this case the difference itself does not introduce a big error in the result ($x_1$ is correct using the first method), but the division amplifies it: $
 \frac{1}{x+\epsilon} \text{ is not similar to } \frac{1}{x}
$
The second algorithm implemented is not stable for $x_2$ for $b>>a, c$. On the other hand the first one could return a wrong result for $x_1$ if $a$ and $c$ are even smaller, compared to $b$, because also the problematic difference become smaller. 

(c) write a function that compute the roots of a quadratic equation accurately in all cases. <br>
I can remove the problematic difference using the first method for $x_2$ and the second one for $x_1$:

In [78]:
def quadratic_equation_best(a, b, c):
    x1 = 2*c/(-b-m.sqrt(b*b-4*a*c))
    x2 = (-b-m.sqrt(b*b-4*a*c))/(2*a) 
    return [x1, x2]

x_12 = quadratic_equation_best(0.001, 1000, 0.001)
print('Solutions: \nx1 = %f \nx2 = %f' %(x_12[0], x_12[1]))

Solutions: 
x1 = -0.000001 
x2 = -999999.999999


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?


In [81]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return x*(x-1)

def fprime(x, accuracy):
    return (f(x+accuracy)-f(x))/accuracy

delta = 10**-2
derivative = fprime(1, delta)
derivatives = np.array([delta, derivative, abs(1-derivative)])

print('\nDelta - Value - Accuracy')
print(derivatives)


Delta - Value - Accuracy
[0.01 1.01 0.01]


Because $\delta$ is too big. In this case, I think is not a problem of stability or poorly conditioned function, but simply the value of $\delta$ considered is too big. Sure the algorithm is not stable (we are taking the difference between very close numbers and dividing it by a small number), and I expect wrong results for much smaller $\delta$, but in this case I would said that the choice of the value of $\delta$ is not the best one.  


(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [82]:
for i in range(2, 8):
    delta = 10**(-2*i)
    derivative = fprime(1, delta)
    derivatives = np.vstack([derivatives, [delta, derivative, abs(1-derivative)]])
    

print('\nDelta - Value - Accuracy')
print(derivatives)


Delta - Value - Accuracy
[[1.00000000e-02 1.01000000e+00 1.00000000e-02]
 [1.00000000e-04 1.00010000e+00 9.99999999e-05]
 [1.00000000e-06 1.00000100e+00 9.99917733e-07]
 [1.00000000e-08 1.00000000e+00 3.92252875e-09]
 [1.00000000e-10 1.00000008e+00 8.28403710e-08]
 [1.00000000e-12 1.00008890e+00 8.89005833e-05]
 [1.00000000e-14 9.99200722e-01 7.99277837e-04]]


The accuracy decreases a lot until $\delta = 10^{-8}$ and then begins to increase. This is in agreement with my previous answer: a better choice of  $\delta$, for example $\delta = 10^{-8}$, gives a very good result and a much smaller accuracy compared to $\delta = 10^{-2}$, but the instability of the algorithm emerges if we try to use even smaller values.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt{1-x^2} {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?


In [85]:

def Riemann_integral(f, xrange, N):
    h = (xrange[1]-xrange[0])/N
    I = 0
    for i in range(N):
        I += h*f(xrange[0]+h/2+h*i)     # f is evaluated in the middle of each slice
    return I

def func(x):
    return m.sqrt(1-x**2)

true_value = m.pi/2

Int1 = Riemann_integral(func, [-1,1], 100)
err1 = Int1-true_value
print('Area of the semicircle of radius 1:', Int1)
print('Error:', err1)


Area of the semicircle of radius 1: 1.5712827762297958
Error: 0.00048644943489928316


The error is not negligible, comparing the value computed with $N=100$ and the true one. 


(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [86]:
# COMPUTATION TIME < 1 second
Int2 = Riemann_integral(func, [-1,1], 1000000)
print('Area of the semicircle of radius 1:', Int2)

Area of the semicircle of radius 1: 1.570796327282045


In [31]:
# COMPUTATION TIME < 1 minute
Int3 = Riemann_integral(func, [-1,1], 50000000)
print('Area of the semicircle of radius 1:', Int3)

Area of the semicircle of radius 1: 1.5707963267966314


In [87]:
# ERROR:
err2 = Int2-true_value
err3 = Int3-true_value

print('ERRORS:')
print('N = 100:', err1)
print('N = 1000000:', err2)
print('N = 50000000:', err3)

ERRORS:
N = 100: 0.00048644943489928316
N = 1000000: 4.871485437263345e-10
N = 50000000: 1.7348344982792696e-12


It is not convenient to run the algorithm for 1 minute. In fact, the error is already very small using $N=1000000$, so there is no need to further increase $N$, also because the gain is not so relevant with respect to the computation time required. 